## LLM Compressor Workbench -- Getting Started

This notebook will demonstrate how common [LLM Compressor](https://github.com/vllm-project/llm-compressor) flows can be run on the [opendatahub/llmcompressor-workbench](https://quay.io/repository/opendatahub/llmcompressor-workbench) image.

We will show how a user can compress and evaluate a Large Language Model, first without data and then with a calibration dataset.

The notebook will detect if a GPU is available. If one is not available, it will demonstrate an abbreviated run, so users without GPU access can still get a feel for `llm-compressor`.


<div class="alert alert-block alert-info">
<b>Note:</b> If you are not using the Workbench image, just be sure to have lm_eval>=0.4.8 and llmcompressor>=0.5.1 installed
</div>

### 1\) Data-Free Model Compression

In [1]:
!pip install llmcompressor lm-eval vllm --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codeflare-sdk 0.26.0 requires pydantic<2, but you have pydantic 2.11.7 which is incompatible.
codeflare-sdk 0.26.0 requires ray[data,default]==2.35.0, but you have ray 2.47.0 which is incompatible.
kfp 2.9.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.5 which is incompatible.
kfp-kubernetes 1.4.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.5 which is incompatible.
kfp-pipeline-spec 0.4.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.5 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# !pip install -qU transformers  --quiet
!pip install transformers==4.51.3 --quiet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from transformers import modeling_utils
if not hasattr(modeling_utils, "ALL_PARALLEL_STYLES") or modeling_utils.ALL_PARALLEL_STYLES is None:
    modeling_utils.ALL_PARALLEL_STYLES = ["tp", "none","colwise",'rowwise']

In [4]:
import torch
use_gpu = torch.cuda.is_available()

In [5]:
from llmcompressor.modifiers.quantization import QuantizationModifier

# model to compress
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# This recipe will quantize all Linear layers except those in the `lm_head`,
#  which is often sensitive to quantization. The W4A16 scheme compresses
#  weights to 4-bit integers while retaining 16-bit activations.
recipe = QuantizationModifier(
    targets="Linear", scheme="W4A16", ignore=["lm_head"]
)

In [6]:
# Load up model using huggingface API
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="auto", 
    torch_dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [7]:
# Run compression using `oneshot`
from llmcompressor import oneshot

model = oneshot(model=model, recipe=recipe, tokenizer=tokenizer)

2025-06-17T06:51:06.156422+0000 | reset | INFO - Compression lifecycle reset
2025-06-17T06:51:06.157677+0000 | from_modifiers | INFO - Creating recipe from modifiers


manager stage: Modifiers initialized


2025-06-17T06:51:07.647286+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers


manager stage: Modifiers finalized


2025-06-17T06:51:07.648231+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers
2025-06-17T06:51:07.648731+0000 | post_process | WARNING - Optimized model is not saved. To save, please provide`output_dir` as input arg.Ex. `oneshot(..., output_dir=...)`


In [8]:
# Save model and tokenizer
model_dir = "./" + model_id.split("/")[-1] + "-W4A16"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir);

2025-06-17T06:51:07.652938+0000 | save_pretrained_wrapper | INFO - Fetching state_dict - this may take some time
2025-06-17T06:51:09.191813+0000 | save_pretrained_wrapper | INFO - Fetching compressor
2025-06-17T06:51:09.192585+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Quantized Compression: 100%|██████████| 509/509 [00:04<00:00, 110.76it/s]

2025-06-17T06:51:13.793994+0000 | save_pretrained_wrapper | INFO - Saving compressed model to disk


### 2\) Evaluate compressed model using open-source `lm_eval` framework

We will evaluate the performance of the model on the [`wikitext`](https://paperswithcode.com/dataset/wikitext-2) language modeling dataset

In [9]:
import lm_eval
from lm_eval.utils import make_table

results = lm_eval.simple_evaluate(
    model="vllm" if use_gpu else "hf",
    model_args={
        "pretrained": model_dir,
        "add_bos_token": True,
        "device": "auto"
    },
    tasks=["wikitext"],
    batch_size="auto" if use_gpu else 4,
    limit=None if use_gpu else 4,
)

INFO 06-17 06:51:20 [__init__.py:243] Automatically detected platform cuda.
INFO 06-17 06:51:22 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-17 06:51:22 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-17 06:51:22 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
INFO 06-17 06:51:31 [config.py:793] This model supports multiple tasks: {'score', 'embed', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.


/opt/app-root/lib64/python3.11/site-packages/llmcompressor/pytorch/__init__.py:19: UserWarning: torch.compile is not supported by llmcompressor for torch 2.0.x
  warnings.warn(


INFO 06-17 06:51:32 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 06-17 06:51:32 [utils.py:2531] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 06-17 06:51:37 [__init__.py:243] Automatically detected platform cuda.
INFO 06-17 06:51:40 [core.py:438] Waiting for init message from front-end.
INFO 06-17 06:51:40 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-17 06:51:40 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-17 06:51:40 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
INFO 06-17 06:51:40 [core.py:65] Initializing a V1 LLM engine (v0.9.0.1) with config: model='./TinyLlama-1.1

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  6.73it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  6.72it/s]



INFO 06-17 06:51:41 [default_loader.py:280] Loading weights took 0.18 seconds
INFO 06-17 06:51:42 [gpu_model_runner.py:1549] Model loading took 0.7432 GiB and 0.398142 seconds
INFO 06-17 06:51:48 [backends.py:459] Using cache directory: /opt/app-root/src/.cache/vllm/torch_compile_cache/e527347c1c/rank_0_0 for vLLM's torch.compile
INFO 06-17 06:51:48 [backends.py:469] Dynamo bytecode transform time: 6.75 s
INFO 06-17 06:51:51 [backends.py:158] Cache the graph of shape None for later use
INFO 06-17 06:52:13 [backends.py:170] Compiling a graph for general shape takes 24.19 s
INFO 06-17 06:52:25 [monitor.py:33] torch.compile takes 30.93 s in total
INFO 06-17 06:52:26 [kv_cache_utils.py:637] GPU KV cache size: 839,248 tokens
INFO 06-17 06:52:26 [kv_cache_utils.py:640] Maximum concurrency for 2,048 tokens per request: 409.79x
INFO 06-17 06:52:49 [gpu_model_runner.py:1933] Graph capturing finished in 23 secs, took 0.37 GiB
INFO 06-17 06:52:49 [core.py:167] init engine (profile, create kv cach

...

Processed prompts:   0%|          | 0/62 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Running loglikelihood requests:   0%|          | 0/62 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/62 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/62 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Running loglikelihood requests: 100%|██████████| 62/62 [00:05<00:00, 11.01it/s]
[rank0]:[W617 06:53:23.078732241 ProcessGroupNCCL.cpp:1476] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [10]:
print(make_table(results))

| Tasks  |Version|Filter|n-shot|    Metric     |   | Value |   |Stderr|
|--------|------:|------|-----:|---------------|---|------:|---|------|
|wikitext|      2|none  |     0|bits_per_byte  |↓  | 0.7583|±  |   N/A|
|        |       |none  |     0|byte_perplexity|↓  | 1.6916|±  |   N/A|
|        |       |none  |     0|word_perplexity|↓  |16.6245|±  |   N/A|



### 3\) Calibrated Compression with a Dataset

Some more advanced compression algorithms require a small dataset of calibration samples that are meant to be a representative random subset of the language the model will see at inference.

We will show how the previous section can be augmented with a calibration dataset and GPTQ, one of the first published LLM compression algorithms.

<div class="alert alert-block alert-info">
<b>Note:</b> This will take several minutes if no GPU is available
</div>

In [11]:
# We will use a new recipe running GPTQ (https://arxiv.org/abs/2210.17323)
# to reduce error caused by quantization. GPTQ requires a calibration dataset.
from llmcompressor.modifiers.quantization import GPTQModifier

recipe = GPTQModifier(targets="Linear", scheme="W4A16", ignore=["lm_head"])

In [12]:
from datasets import load_dataset

# Create the calibration dataset, using Huggingface datasets API
dataset_id = "HuggingFaceH4/ultrachat_200k"

# Select number of samples. 512 samples is a good place to start.
# Increasing the number of samples can improve accuracy.
num_calibration_samples = 512 if use_gpu else 4
max_sequence_length = 2048 if use_gpu else 16

# Load dataset
ds = load_dataset(dataset_id, split="train_sft")
# Shuffle and grab only the number of samples we need
ds = ds.shuffle(seed=42).select(range(num_calibration_samples))

# Preprocess and tokenize into format the model uses
def preprocess(example):
    text = tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
        )
    return tokenizer(
        text,
        padding=False,
        max_length=max_sequence_length,
        truncation=True,
        add_special_tokens=False,
    )

ds = ds.map(preprocess, remove_columns=ds.column_names)

Map:   0%|          | 0/512 [00:00<?, ? examples/s]

In [13]:
# oneshot modifies model in-place, so reload
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype="auto"
)
# run oneshot again, with dataset
model = oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=max_sequence_length,
    num_calibration_samples=num_calibration_samples,
)

2025-06-17T06:53:28.626297+0000 | reset | INFO - Compression lifecycle reset
2025-06-17T06:53:28.627428+0000 | from_modifiers | INFO - Creating recipe from modifiers
2025-06-17T06:53:28.629109+0000 | _check_build_quant_modifier | WARNING - GPTQ quantization is set to True without an active quantization modifier.
2025-06-17T06:53:28.629511+0000 | _build_quant_modifier | INFO - Building quantization modifier with args: {'targets': 'Linear', 'scheme': 'W4A16', 'ignore': ['lm_head']}
2025-06-17T06:53:28.678451+0000 | _check_calibration_data | INFO - Skipping QuantizationModifier calibration, it is not required for the provided quantization config.


(1/23): Calibrating: 100%|██████████| 512/512 [00:07<00:00, 68.21it/s]

2025-06-17T06:53:38.555433+0000 | on_sequential_batch_end | INFO - Quantizing model.layers.0.self_attn.q_proj using 512 samples


2025-06-17T06:53:39.554321+0000 | compress | METRIC - time 1.00s
2025-06-17T06:53:39.555133+0000 | compress | METRIC - error 611.05
2025-06-17T06:53:39.556127+0000 | compress | METRIC - GPU 0 | usage: 10.51% | total memory: 24 GB
2025-06-17T06:53:39.556507+0000 | compress | METRIC - GPU 1 | usage: 8.54% | total memory: 24 GB
2025-06-17T06:53:39.556886+0000 | compress | METRIC - GPU 2 | usage: 8.54% | total memory: 24 GB
2025-06-17T06:53:39.557254+0000 | compress | METRIC - GPU 3 | usage: 7.36% | total memory: 24 GB
2025-06-17T06:53:39.557638+0000 | compress | METRIC - Compressed module size: 8.486912 MB
2025-06-17T06:53:39.558644+0000 | on_sequential_batch_end | INFO - Quantizing model.layers.0.self_attn.k_proj using 512 samples
2025-06-17T06:53:40.514340+0000 | compress | METRIC - time 0.96s
2025-06-17T06:53:40.514954+0000 | compress | METRIC - error 595.48
2025-06-17T06:53:40.515827+0000 | compress | METRIC - GPU 0 | usage: 10.51% | total memory: 24 GB
2025-06-17T06:53:40.516178+0000

(2/23): Calibrating: 100%|██████████| 512/512 [00:07<00:00, 69.11it/s]

...

2025-06-17T07:00:23.243706+0000 | compress | METRIC - time 1.00s
2025-06-17T07:00:23.244485+0000 | compress | METRIC - error 6350.40
2025-06-17T07:00:23.245308+0000 | compress | METRIC - GPU 0 | usage: 11.04% | total memory: 24 GB
2025-06-17T07:00:23.245679+0000 | compress | METRIC - GPU 1 | usage: 12.17% | total memory: 24 GB
2025-06-17T07:00:23.246086+0000 | compress | METRIC - GPU 2 | usage: 12.17% | total memory: 24 GB
2025-06-17T07:00:23.246494+0000 | compress | METRIC - GPU 3 | usage: 11.04% | total memory: 24 GB
2025-06-17T07:00:23.246913+0000 | compress | METRIC - Compressed module size: 8.486912 MB
2025-06-17T07:00:23.247953+0000 | on_sequential_batch_end | INFO - Quantizing model.layers.21.self_attn.k_proj using 512 samples
2025-06-17T07:00:24.241940+0000 | compress | METRIC - time 0.99s
2025-06-17T07:00:24.243078+0000 | compress | METRIC - error 2215.37
2025-06-17T07:00:24.244073+0000 | compress | METRIC - GPU 0 | usage: 11.04% | total memory: 24 GB
2025-06-17T07:00:24.24450

(23/23): Propagating: 100%|██████████| 512/512 [00:01<00:00, 292.62it/s]
manager stage: Modifiers initialized


2025-06-17T07:00:37.414273+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers


manager stage: Modifiers finalized


2025-06-17T07:00:37.415971+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers
2025-06-17T07:00:37.416480+0000 | post_process | WARNING - Optimized model is not saved. To save, please provide`output_dir` as input arg.Ex. `oneshot(..., output_dir=...)`


In [14]:
# Save model and tokenizer
model_dir = "./" + model_id.split("/")[-1] + "-GPTQ-W4A16"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir);

2025-06-17T07:00:37.424784+0000 | save_pretrained_wrapper | INFO - Fetching state_dict - this may take some time
2025-06-17T07:00:39.437497+0000 | save_pretrained_wrapper | INFO - Fetching compressor
2025-06-17T07:00:39.438306+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Quantized Compression: 100%|██████████| 509/509 [00:04<00:00, 107.63it/s]

2025-06-17T07:00:44.172957+0000 | save_pretrained_wrapper | INFO - Saving compressed model to disk


### 4\) Rerun `lm_eval`

Note that perplexity score has improved (lower is better) for this `TinyLlama` model. 

In [15]:
results = lm_eval.simple_evaluate(
    model="vllm" if use_gpu else "hf",
    model_args={
        "pretrained": model_dir,
        "add_bos_token": True,
        "device": "auto"
    },
    tasks=["wikitext"],
    batch_size="auto" if use_gpu else 4,
    limit=None if use_gpu else 4,
)

INFO 06-17 07:00:45 [config.py:793] This model supports multiple tasks: {'score', 'embed', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.
INFO 06-17 07:00:45 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-17 07:00:50 [__init__.py:243] Automatically detected platform cuda.
INFO 06-17 07:00:53 [core.py:438] Waiting for init message from front-end.
INFO 06-17 07:00:53 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-17 07:00:53 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-17 07:00:53 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
INFO 06-17 07:00:53 [core.py:65] Initializing a V1 LLM engine (v0.9.0.1) with config: model='./TinyLlama-1.1B-Chat-v1.0-GPTQ-W4A16', speculative_config=None, tokenizer='./TinyLlama-1.1B-Chat-v1.0-GPTQ-W4A16', skip_tokenizer_init=False, 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.37it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.37it/s]



INFO 06-17 07:00:54 [default_loader.py:280] Loading weights took 0.22 seconds
INFO 06-17 07:00:55 [gpu_model_runner.py:1549] Model loading took 0.7432 GiB and 0.488780 seconds
INFO 06-17 07:01:01 [backends.py:459] Using cache directory: /opt/app-root/src/.cache/vllm/torch_compile_cache/f2259db575/rank_0_0 for vLLM's torch.compile
INFO 06-17 07:01:01 [backends.py:469] Dynamo bytecode transform time: 6.70 s
INFO 06-17 07:01:04 [backends.py:158] Cache the graph of shape None for later use
INFO 06-17 07:01:26 [backends.py:170] Compiling a graph for general shape takes 23.90 s
INFO 06-17 07:01:38 [monitor.py:33] torch.compile takes 30.61 s in total
INFO 06-17 07:01:39 [kv_cache_utils.py:637] GPU KV cache size: 813,920 tokens
INFO 06-17 07:01:39 [kv_cache_utils.py:640] Maximum concurrency for 2,048 tokens per request: 397.42x
INFO 06-17 07:02:01 [gpu_model_runner.py:1933] Graph capturing finished in 22 secs, took 0.37 GiB
INFO 06-17 07:02:01 [core.py:167] init engine (profile, create kv cach

[Task: wikitext] metric word_perplexity is defined, but aggregation is not. using default aggregation=weighted_perplexity
[Task: wikitext] metric word_perplexity is defined, but higher_is_better is not. using default higher_is_better=False
[Task: wikitext] metric byte_perplexity is defined, but aggregation is not. using default aggregation=weighted_perplexity
[Task: wikitext] metric byte_perplexity is defined, but higher_is_better is not. using default higher_is_better=False
[Task: wikitext] metric bits_per_byte is defined, but aggregation is not. using default aggregation=bits_per_byte
[Task: wikitext] metric bits_per_byte is defined, but higher_is_better is not. using default higher_is_better=False
Running loglikelihood requests:   0%|          | 0/62 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/62 [00:00<?, ?it/s]

...

Processed prompts:   0%|          | 0/62 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Running loglikelihood requests:   0%|          | 0/62 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/62 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/62 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Running loglikelihood requests: 100%|██████████| 62/62 [00:05<00:00, 10.46it/s]
[rank0]:[W617 07:02:35.790416464 ProcessGroupNCCL.cpp:1476] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [16]:
print(make_table(results))

| Tasks  |Version|Filter|n-shot|    Metric     |   | Value |   |Stderr|
|--------|------:|------|-----:|---------------|---|------:|---|------|
|wikitext|      2|none  |     0|bits_per_byte  |↓  | 0.7508|±  |   N/A|
|        |       |none  |     0|byte_perplexity|↓  | 1.6827|±  |   N/A|
|        |       |none  |     0|word_perplexity|↓  |16.1636|±  |   N/A|

